In [ ]:
import requests
from zipfile import ZipFile
from io import BytesIO
import os
import cv2
from matplotlib import pyplot as plt
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Input,Concatenate
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionV3, ResNet50, Xception, MobileNetV2, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

# **DOWNLOADING DATASET**

In [ ]:
url = 'https://archive.ics.uci.edu/static/public/773/defungi.zip'
response = requests.get(url)
if response.status_code == 200:
    print("Download successful.")
else:
    print("Failed to download the file. Status code:", response.status_code)

# GOOGLE DRIVE

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
if response.status_code == 200:
    zip_file = ZipFile(BytesIO(response.content))
    zip_file.extractall('/content/dataset')
    print("Extraction complete.")
    zip_file.close()

In [ ]:
datadirectory = '/content/dataset'

categories=['H1','H2','H3','H5','H6']

for category in categories:
    path = os.path.join(datadirectory, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
        plt.imshow(img_array, cmap='gray')
        plt.show()
        break
    break

In [ ]:
print(img_array.shape)

# DATA PREPROCESSING 1ST EXPERIMENT



*   The Data is prepared with resoultion of 80x80



In [ ]:
img_size = 80

In [ ]:
new_array_trail = cv2.resize(img_array,(img_size,img_size))
plt.imshow(new_array_trail, cmap='gray')
plt.show()

In [ ]:
training_data=[]

def create_training_data():
    for category in categories:
        path = os.path.join(datadirectory, category)
        class_num=categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array =cv2.resize(img_array,(80,80))
                training_data.append([new_array,class_num])
            except Exception as e:
                pass
create_training_data()

In [ ]:
len(training_data)

In [ ]:
random.shuffle(training_data)

In [ ]:
X = []
y = []

In [ ]:
for features, labels in training_data:
    X.append(features)
    y.append(labels)

X = np.array(X).reshape(-1, 80,80, 1)
y= np.array(y)

In [ ]:

y = tf.keras.utils.to_categorical(y, num_classes=5)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:

path_X = '/content/drive/My Drive/X.pickle'
path_y = '/content/drive/My Drive/y.pickle'

# Saving X
with open(path_X, 'wb') as pickle_out:
    pickle.dump(X, pickle_out)

# Saving y
with open(path_y, 'wb') as pickle_out:
    pickle.dump(y, pickle_out)

In [ ]:
path_X = '/content/drive/My Drive/X.pickle'
path_y = '/content/drive/My Drive/y.pickle'

In [ ]:
with open(path_X, 'rb') as pickle_in:
    X = pickle.load(pickle_in)


In [ ]:
with open(path_y, 'rb') as pickle_in:
    y = pickle.load(pickle_in)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Testing data shape:", X_test.shape)
print("Testing labels shape:", y_test.shape)

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

# **Basic ANN**

In [ ]:
input_shape = (80, 80, 1)
num_classes = 5

In [ ]:
model_1 = Sequential([
    Flatten(input_shape=input_shape),
    Dense(512, activation='relu'),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [ ]:
model_1.summary()

In [ ]:
model_1.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
checkpoint = ModelCheckpoint('best_model_1.h5', save_best_only=True, monitor='val_accuracy', mode='max')


history = model_1.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    callbacks=[checkpoint]
)

plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
best_model = tf.keras.models.load_model('best_model_1.h5')

test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

# **Dense ANN**

In [ ]:
model_1_1 = Sequential([
    Flatten(input_shape=input_shape),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])


In [ ]:
model_1_1.compile(
    optimizer=Adam(),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
checkpoint = ModelCheckpoint('best_model_1_1.h5', save_best_only=True, monitor='val_accuracy', mode='max')

history = model_1_1.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    callbacks=[checkpoint]
)
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
best_model = tf.keras.models.load_model('best_model_1_1.h5')

test_loss, test_accuracy = best_model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

# **CNN**

In [ ]:
model_2 = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(num_classes, activation='softmax')
])

In [ ]:
model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_2.summary()

In [ ]:
checkpoint_2 = ModelCheckpoint('best_model_2.h5', save_best_only=True, monitor='val_accuracy', mode='max')

history = model_2.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=20,
    batch_size=64,
    callbacks=[checkpoint_2]
)
plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
best_model_2 = tf.keras.models.load_model('best_model_2.h5')
# Evaluate the model on the test data
test_loss, test_accuracy = best_model_2.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)

# **DATA PREPROCESSING FOR HIGH RESOLUTION IMAGE OF 80x80**

In [ ]:
orginal_training_data=[]

def create_training_data():
    for category in categories:
        path = os.path.join(datadirectory, category)  # Use category here, not categories
        class_num=categories.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_img_array =cv2.resize(img_array,(80,80))
                orginal_training_data.append([new_img_array,class_num])
            except Exception as e:
                pass
create_training_data()

In [ ]:
orginal_training_data

In [ ]:
random.shuffle(orginal_training_data)

In [ ]:
new_array_trail = cv2.resize(img_array,(80,80))
plt.imshow(new_array_trail, cmap='gray')
plt.show()

In [ ]:
X_new=[]
y_new=[]

In [ ]:
for features, labels in orginal_training_data:
    X_new.append(features)
    y_new.append(labels)

X_new = np.array(X_new).reshape(-1, 80,80, 1)
y_new= np.array(y_new)

In [ ]:
y_new = tf.keras.utils.to_categorical(y_new, num_classes=5)

In [ ]:
path_X_new = '/content/drive/My Drive/X_new.pickle'
path_y_new = '/content/drive/My Drive/y_new.pickle'

with open(path_X_new, 'wb') as pickle_out:
    pickle.dump(X_new, pickle_out)

with open(path_y_new, 'wb') as pickle_out:
    pickle.dump(y_new, pickle_out)

In [ ]:
path_X_new = '/content/drive/MyDrive/X_new.pickle'
path_y_new = '/content/drive/My Drive/y_new.pickle'

In [ ]:
with open(path_X_new, 'rb') as pickle_in:
    X_new = pickle.load(pickle_in)

In [ ]:
with open(path_y_new, 'rb') as pickle_in:
    y_new = pickle.load(pickle_in)

In [ ]:
Xnew_train, Xnew_test, ynew_train, ynew_test = train_test_split(X_new, y_new, test_size=0.2, random_state=42)

print("Training data shape:", Xnew_train.shape)
print("Training labels shape:", ynew_train.shape)
print("Testing data shape:", Xnew_test.shape)
print("Testing labels shape:", ynew_test.shape)

# **IMAGEDATAGENERATOR FOR PREPROCESSING**


In [ ]:
Xnew_train_1, Xnew_val, ynew_train_1, ynew_val = train_test_split(Xnew_train, ynew_train, test_size=0.2, random_state=42)

In [ ]:
print("Training data shape:", Xnew_train_1.shape)
print("Training labels shape:", ynew_train_1.shape)
print("validation data shape:", Xnew_val.shape)
print("validation labels shape:", ynew_val.shape)
print("Testing data shape:", Xnew_test.shape)
print("Testing labels shape:", ynew_test.shape)

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen=ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_generator = train_datagen.flow(
    Xnew_train_1, ynew_train_1,
    batch_size=32
)

validation_generator = test_datagen.flow(
    Xnew_val, ynew_val,
    batch_size=32
)
test_generator = test_datagen.flow(
    Xnew_test, ynew_test,
    batch_size=32
)


In [ ]:
train_generator

# **CNN ARCHITECTURE WITH ADDITIONAL CONVOLUTION AND DENSE LAYERS**

In [ ]:
model_3 = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(80, 80, 1)),
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu'),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    Flatten(),

    Dense(1024, activation='relu'),
    Dropout(0.5),
    Dense(512, activation='relu'),
    Dropout(0.5),

    Dense(5, activation='softmax')
])

model_3.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
model_3.summary()

In [ ]:
checkpoint_cb = ModelCheckpoint(
    'best_model_3.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)
early_stopping = EarlyStopping(monitor='val_accuracy', patience=2, min_delta=0.005, mode='max', verbose=1)

In [ ]:
history = model_3.fit(
    train_generator,
    epochs=20,
    batch_size=32,
    validation_data=(Xnew_val, ynew_val),
    callbacks=[checkpoint_cb, early_stopping],
    verbose=1
)

plt.figure(figsize=(10, 5))
plt.plot(history.history['accuracy'], label='Train')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()


In [ ]:
model_3.save('final_model_complete')

In [ ]:
best_model_3= tf.keras.models.load_model('final_model_complete')
test_loss, test_accuracy = best_model_3.evaluate(Xnew_test, ynew_test)
print("Test Accuracy:", test_accuracy)

# **Advanced arcitectures**

In [ ]:
def create_model(model_name):
    if model_name == 'InceptionV3':
        base_model = InceptionV3(include_top=False, weights=None, input_tensor=Input(shape=(80, 80, 1)))
    elif model_name == 'ResNet50':
        base_model = ResNet50(include_top=False, weights=None, input_tensor=Input(shape=(80, 80, 1)))
    elif model_name == 'Xception':
        base_model = Xception(include_top=False, weights=None, input_tensor=Input(shape=(80, 80, 1)))
    elif model_name == 'MobileNetV2':
        base_model = MobileNetV2(include_top=False, weights=None, input_tensor=Input(shape=(80, 80, 1)))
    elif model_name == 'DenseNet121':
        base_model = DenseNet121(include_top=False, weights=None, input_tensor=Input(shape=(80, 80, 1)))
    else:
        return None

    base_model.trainable = False
    pool1 = GlobalAveragePooling2D()(base_model.output)
    Dense1= Dense(512, activation='relu')(pool1)
    Dropout1=Dropout(0.3)(Dense1)
    Dense2=Dense(256, activation='relu')(Dropout1)
    Normalizer1=BatchNormalization()(Dense2)
    Dense3= Dense(128, activation='relu')(Normalizer1)
    Dropout2=Dropout(0.3)(Dense3)
    predictions = Dense(5, activation='softmax')(Dropout2)
    model = Model(inputs=base_model.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model


# **INCEPTIONV3**

In [ ]:
inception = create_model('InceptionV3')

In [ ]:
inception.summary()

In [ ]:
checkpoint_inception = ModelCheckpoint(
    'best_model_inception.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max',
    verbose=1
)

In [ ]:
history_inception = inception.fit(
    train_generator,
    epochs=1,
    batch_size=32,
    validation_data=(Xnew_val, ynew_val),
    callbacks=[checkpoint_inception],
    verbose=1
)


# **Advanced Architectures comparision**

In [ ]:
advanced_architectures= ['InceptionV3', 'ResNet50', 'Xception', 'MobileNetV2', 'DenseNet121']

In [ ]:
model_metrics = {}

for md in advanced_architectures:
  model_metrics[md] = {
        'train_loss': [],
        'train_accuracy': [],
        'val_loss': [],
        'val_accuracy': [],
        'test_loss': None,
        'test_accuracy': None
    }

In [ ]:
for md in advanced_architectures:
    adv_model = create_model(md)
    if adv_model is not None:
        print(f"training {md}")
        checkpoint_path = f'best_model_{md}.h5'
        checkpoint_cb = ModelCheckpoint(
            checkpoint_path,
            save_best_only=True,
            monitor='val_accuracy',
            mode='max',
            verbose=1
        )
        early_stopping = EarlyStopping(
            monitor='val_accuracy',
            patience=3,
            min_delta=0.01,
            mode='max',
            verbose=1
        )
        history = adv_model.fit(
            train_generator,
            epochs=20,
            validation_data=validation_generator,
            callbacks=[checkpoint_cb, early_stopping],
            verbose=1
        )

        plt.figure(figsize=(10, 5))
        plt.plot(history.history['accuracy'], label='Train')
        plt.plot(history.history['val_accuracy'], label='Validation')
        plt.title('Model Accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(loc='upper left')
        plt.show()


        final_model_path = f'final_model_{md}_complete.h5'
        adv_model.save(final_model_path)
        test_loss, test_accuracy = adv_model.evaluate(test_generator)

        model_metrics[md]['train_loss'] = history.history['loss']
        model_metrics[md]['train_accuracy'] = history.history['accuracy']
        model_metrics[md]['val_loss'] = history.history['val_loss']
        model_metrics[md]['val_accuracy'] = history.history['val_accuracy']
        model_metrics[md]['test_loss'] = test_loss
        model_metrics[md]['test_accuracy'] = test_accuracy


In [ ]:
model_metrics